## PYMUPDF

In [127]:
import os
# os.environ['API_KEY'] = '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'
# os.environ['API_KEY'] = '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'
# os.environ['API_KEY'] = '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'
os.environ['API_KEY'] = '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'

### Helper Functions

In [128]:
import os
import csv
import json
from typing import List, Dict, Any
import fitz  # PyMuPDF
from groq import Groq
import pandas as pd
import chardet

# Initialize Groq client
client = Groq(api_key=os.getenv("API_KEY"))

def groq_response(question):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
        model="llama-3.1-70b-versatile",
        temperature=0,
        max_tokens=128,
    )
    return response.choices[0].message.content


def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text from a PDF file."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF {pdf_path}: {str(e)}")
        return ""

def read_file_with_auto_detect(file_path):
    """Read a file with automatic encoding detection."""
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    
    detected = chardet.detect(raw_data)
    encodings = [detected['encoding'], 'utf-8', 'latin-1', 'ascii']
    
    for encoding in encodings:
        try:
            return raw_data.decode(encoding)
        except UnicodeDecodeError:
            continue
    
    # If all else fails, return the raw bytes as a string
    return str(raw_data)


def detect_circular_endorsements(user_id: str, recommender_ids: List[str], endorsement_data: Dict[str, List[str]]) -> List[str]:
    """Detect circular endorsements."""
    circular_endorsements = []
    for recommender_id in recommender_ids:
        if recommender_id in endorsement_data:
            if user_id in endorsement_data[recommender_id]:
                circular_endorsements.append(recommender_id)
    return circular_endorsements

def load_endorsement_data(csv_path: str) -> Dict[str, List[str]]:
    """Load endorsement data from CSV file."""
    endorsement_data = {}
    with open(csv_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            user_id, recommenders = row
            endorsement_data[user_id] = json.loads(recommenders.replace("'", '"'))
    return endorsement_data


In [3]:
# Base directories
resumes_dir = "Dataset/Final_Resumes"
recommendations_base_dir = "Dataset/Final_Recommendation_Letters"


In [4]:
recommendation_texts = {}

In [5]:
endorsement_data = load_endorsement_data("./Dataset/Final_Persons_And_Recommenders.csv")

### Extract Recommendation text for each candidate

In [7]:
for user_id in endorsement_data.keys():
    recommendation_dir = os.path.join(recommendations_base_dir, f"Recommendation_Letters_of_ID_{user_id}")
    
    recommendation_paths = {f.split('.')[0].split('_')[-1]:os.path.join(recommendation_dir, f) for f in os.listdir(recommendation_dir) if f.endswith('.txt')}
    recommendation_text = {key:read_file_with_auto_detect(path) for key,path in recommendation_paths.items()}
    
    recommendation_texts[user_id] = recommendation_text

### Extract Resume text for each user using PYMUPDF

In [8]:
resume_texts = {}
for user_id in [str(i) for i in range(1000)]:
    resume_path = os.path.join(resumes_dir, f"Resume_of_ID_{user_id}.pdf")
    if os.path.exists(resume_path):
        resume_text = extract_text_from_pdf(resume_path)
    else:
        resume_text = ""
    resume_texts[user_id] = resume_text

In [9]:
len(resume_texts)

1000

In [10]:
recommendation_texts

{'0': {'391': "Having worked as an 8th grade Language Arts teacher and collaborated with various teaching professionals, I wholeheartedly recommend the bilingual language arts sixth-grade teacher for any educational role. With over four years of dedicated teaching experience, this professional demonstrates a strong commitment to fostering positive relationships with students, parents, and colleagues. Their innovative lesson planning and ability to implement engaging teaching strategies, such as utilizing educational technology, are impressive.\r\n\r\nMoreover, their participation in intervention planning and community programs showcases a proactive approach to addressing the diverse needs of students. Their bilingual capabilities further enhance their value in today’s multicultural classroom environment. The ability to work collaboratively across departments and their involvement in school improvement initiatives illustrate a deep commitment to educational excellence. I have no doubt t

### Use pre-extracted resume summaries using GROQ and extract them into a list

In [85]:
import pandas as pd
df = pd.read_csv("resume_summaries.csv")

resume_summaries = df['Resume Summary'].tolist()

In [86]:
output = pd.DataFrame(columns=['ID', 'Resume Summary', 'Recommendation Letter Score', 'Resume Score', 'Suspicious Wording Score'])

In [87]:
output['ID'] = [str(i) for i in range(1000)]
output['Resume Summary'] = resume_summaries

In [90]:
print(resume_summaries[85])

1. Resume Summary: 
John Chriskey is a seasoned Purchaser/Production Coordinator with 15 years of experience in the Aviation industry, specializing in procurement, logistics, and inventory management. He has worked with various aircraft makes and models, including Boeing, Airbus, and Lockheed.

2. Objective: 
Seeking a challenging role in procurement, logistics, or supply chain management where he can utilize his skills and experience to contribute to a progressive organization.

3. Experience: 
- Purchaser/Production Coordinator at Flightstar (January 2011 - Current): Responsible for purchasing, inventory management, and production coordination. 
- Purchasing Clerk/Material Specialist at Vision Airlines (February 2009 - December 2010): Managed purchasing, stores, warehouse, shipping, and receiving operations.
- Purchasing Agent/Material Expeditor (February 2005 - January 2009): Coordinated purchasing, expediting, and logistics for various airline companies and military organizations.


In [104]:
resume_summaries[0]

'1. Resume Summary: Dedicated and enthusiastic educator with over four years of experience in education, specializing in bilingual language arts and possessing strong communication skills to build trust among students, parents, and administrators.\n\n2. Objective: None.\n\n3. Experience: \n- Bilingual Language Arts Sixth Grade Teacher at Company Name (2014-Present): Developed and implemented interactive learning mediums, participated in intervention plans, and implemented educational technology programs.\n- Bilingual Substitute Teacher at Company Name (2014): Worked in elementary schools, provided support for teachers on leave, and substituted in various classroom programs.\n- Bilingual 4th Grade Teacher at Company Name (2013-2014): Developed and implemented lesson plans, analyzed data, and cooperated with staff in planning instructional goals.\n- Student Development Assistant at Company Name (2011-2012): Coordinated community tours, created data reports, and collaborated with the FWIS

### Use pre-defined prompts to grade each recommendation letter, resume based on recommendation letters, find suspicious wording

In [ ]:
from Prompts import prompt_for_grading_recommendation_letter, prompt_for_resume_summary, prompt_for_resume_score_based_on_recommendation, prompt_for_suspicious_wording
for i in range(107,1000):
    user_id = str(i)
    candidate_experience = resume_summaries[int(user_id)].split('3. Experience:')[1].split('4. Accomplishments:')[0]
    candidate_summary = resume_summaries[int(user_id)].split('1. Resume Summary:')[1].split('2. Objective:')[0]
    if user_id in endorsement_data:
        recommendation_score = 0
        for recommender_id, recommendation_letter in recommendation_texts[user_id].items():
            resume_recommender = resume_summaries[int(recommender_id)].split('1. Resume Summary:')[1].split('2. Objective:')[0]
            recommendation_score += int(groq_response(prompt_for_grading_recommendation_letter.format(resume_candidate=resume_summaries[int(user_id)], recommendation_letter=recommendation_letter, resume_recommender=resume_recommender)))
        
        recommendation_score = recommendation_score / len(recommendation_texts[user_id])
        
        resume_score = int(groq_response(prompt_for_resume_score_based_on_recommendation.format(resume_candidate = candidate_experience, recommendation_letters='\n'.join(list(recommendation_texts[user_id].values())))))
        output.loc[output['ID'] == user_id, 'Recommendation Letter Score'] = recommendation_score
        output.loc[output['ID'] == user_id, 'Resume Score based on Recommendations'] = resume_score
    else:
        output.loc[output['ID'] == user_id, 'Recommendation Letter Score'] = None
        output.loc[output['ID'] == user_id, 'Resume Score based on Recommendations'] = -1
    
    output.loc[output['ID'] == user_id, 'Suspicious Wording Score'] = int(groq_response(prompt_for_suspicious_wording.format(resume_text=candidate_summary)))

    print(f"Processed user {user_id}")


In [137]:
output['Resume Score based on Recommendations'].value_counts()

Resume Score based on Recommendations
 8.0    75
 9.0    21
 4.0     5
 6.0     5
 2.0     2
-1.0     2
Name: count, dtype: int64

In [138]:
output.head()

,ID,Resume Summary,Recommendation Letter Score,Resume Score,Suspicious Wording Score,Resume Score based on Recommendations
0,0,1. Resume Summary: Dedicated and enthusiastic ...,7.0,8,8,8.0
1,1,1. Resume Summary: The candidate is a Mortgage...,6.0,8,8,8.0
2,2,1. Resume Summary: Diana Dimas is a fitness pr...,5.6,8,8,8.0
3,3,1. Resume Summary: The candidate is a highly e...,7.0,8,8,8.0
4,4,1. Resume Summary: Virginia Francis is a detai...,7.714286,9,8,9.0


In [43]:
# suspicious_wording_score = groq_response(prompt_for_suspicious_wording.format(resume_text=resume_summary_candidate))

In [44]:
# user_id = '0'
# recommendation_letters_text = "\n".join(recommendation_texts[user_id].values())

In [45]:
# for user_id in [str(i) for i in range(10)]:
#     resume_score = groq_response(prompt_for_resume_score_based_on_recommendation.format(resume_candidate=resume_summary_candidate, recommendation_letters=recommendation_letters_text))

### Initialise a Netwrokx directed graph with each user a node

In [12]:
import networkx as nx
import matplotlib.pyplot as plt

G_recommendations = nx.DiGraph()
G_recommendations.add_nodes_from([str(i) for i in range(1000)])

In [13]:
for user_id, recommender_ids in endorsement_data.items():
    for recommender_id in recommender_ids:
        G_recommendations.add_edge(user_id,str(recommender_id))

In [14]:
list(G_recommendations.successors('3'))

['194', '122']

### Find cycles of size 2-5

In [15]:
# Dictionary to store the number of cycles of different sizes for each node
node_cycle_count = {i: {2: 0, 3: 0, 4: 0, 5: 0} for i in G_recommendations.nodes()}

# Function to find cycles of length 2-5
def find_cycles_of_size_2_to_5(G):
    # Iterate over each node
    for i in G.nodes():
        # For 2-length cycles
        for j in G.successors(i):  # Neighbors that i points to
            if G.has_edge(j, i):  # If j points back to i, we have a 2-cycle
                node_cycle_count[i][2] += 1
        
        # For 3-length cycles
        for j in G.successors(i):  # Neighbors that i points to
            for k in G.successors(j):  # Neighbors that j points to
                if G.has_edge(k, i):  # If k points back to i, we have a 3-cycle
                    node_cycle_count[i][3] += 1
        
        # For 4-length cycles
        for j in G.successors(i):  # Neighbors that i points to
            for k in G.successors(j):  # Neighbors that j points to
                for l in G.successors(k):  # Neighbors that k points to
                    if G.has_edge(l, i):  # If l points back to i, we have a 4-cycle
                        node_cycle_count[i][4] += 1
        
        # For 5-length cycles
        for j in G.successors(i):  # Neighbors that i points to
            for k in G.successors(j):  # Neighbors that j points to
                for l in G.successors(k):  # Neighbors that k points to
                    for m in G.successors(l):  # Neighbors that l points to
                        if G.has_edge(m, i):  # If m points back to i, we have a 5-cycle
                            node_cycle_count[i][5] += 1

# Call the function to find cycles of size 2-5
find_cycles_of_size_2_to_5(G_recommendations)

In [16]:
node_cycle_count

{'0': {2: 0, 3: 0, 4: 0, 5: 0},
 '1': {2: 0, 3: 0, 4: 1, 5: 4},
 '2': {2: 0, 3: 1, 4: 0, 5: 1},
 '3': {2: 1, 3: 1, 4: 2, 5: 6},
 '4': {2: 0, 3: 0, 4: 1, 5: 0},
 '5': {2: 0, 3: 0, 4: 0, 5: 4},
 '6': {2: 0, 3: 0, 4: 0, 5: 0},
 '7': {2: 1, 3: 0, 4: 1, 5: 2},
 '8': {2: 0, 3: 1, 4: 1, 5: 2},
 '9': {2: 0, 3: 0, 4: 1, 5: 1},
 '10': {2: 0, 3: 0, 4: 0, 5: 0},
 '11': {2: 1, 3: 0, 4: 2, 5: 1},
 '12': {2: 0, 3: 0, 4: 0, 5: 0},
 '13': {2: 0, 3: 0, 4: 0, 5: 1},
 '14': {2: 0, 3: 0, 4: 0, 5: 0},
 '15': {2: 0, 3: 0, 4: 3, 5: 3},
 '16': {2: 0, 3: 0, 4: 0, 5: 1},
 '17': {2: 0, 3: 0, 4: 0, 5: 1},
 '18': {2: 0, 3: 0, 4: 0, 5: 4},
 '19': {2: 0, 3: 0, 4: 0, 5: 0},
 '20': {2: 0, 3: 1, 4: 1, 5: 0},
 '21': {2: 0, 3: 0, 4: 1, 5: 0},
 '22': {2: 0, 3: 1, 4: 0, 5: 1},
 '23': {2: 0, 3: 0, 4: 0, 5: 0},
 '24': {2: 0, 3: 0, 4: 2, 5: 3},
 '25': {2: 0, 3: 0, 4: 1, 5: 1},
 '26': {2: 0, 3: 0, 4: 0, 5: 0},
 '27': {2: 0, 3: 1, 4: 3, 5: 6},
 '28': {2: 0, 3: 1, 4: 2, 5: 7},
 '29': {2: 0, 3: 0, 4: 0, 5: 0},
 '30': {2: 0, 3: 0, 

### Find max cycles of each cycle length

In [17]:
max_cycles = {'2':0,'3':0,'4':0,'5':0}
recommendation_redundancy_score = {}
for key, cycles_dict in node_cycle_count.items():
    for cycle_length, count in cycles_dict.items():
        if count > max_cycles[str(cycle_length)]:
            max_cycles[str(cycle_length)] = count

In [18]:
max_cycles

{'2': 2, '3': 3, '4': 7, '5': 25}

### Assign weights in reciprocal to cycle length

In [19]:
weights_for_each_cycle_length = [1/i/sum([1/i for i in range(2,6)]) for i in range(2,6)]
weights_for_each_cycle_length

[0.38961038961038963,
 0.25974025974025977,
 0.19480519480519481,
 0.15584415584415587]

### Calculate the final recommendation redundancy score that tells about the redunducy of a user's all recommendations

In [20]:
recommendation_redundancy_score = {}

for key, cycles_dict in node_cycle_count.items():
    recommendation_redundancy_score[key] = sum([count*w/max_cycles[str(cycle_length)] for (cycle_length, count),w in zip(cycles_dict.items(),weights_for_each_cycle_length)])

In [21]:
max(recommendation_redundancy_score.values())

0.7956709956709958

In [142]:
# output the recommendation redundancy score in a csv file
recommendation_redundancy_score_df = pd.DataFrame(recommendation_redundancy_score.items(), columns=['ID', 'Recommendation Redundancy Score'])
recommendation_redundancy_score_df.to_csv('recommendation_redundancy_score.csv', index=False)


In [139]:
output['Recommendation Redundancy Score'] = recommendation_redundancy_score.values()

In [141]:
output.iloc[:100].to_csv("output.csv",columns=['ID','Resume Summary', 'Resume Score based on Recommendations', 'Suspicious Wording Score', 'Recommendation Redundancy Score'],index=False)

In [33]:
G_recommendations.nodes()

NodeView(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '

### Total categories to consider:
1. Resume <-> Recommendation letter
2. Recommendation letter <-> Resume
3. Suspicious words
4. circular recommendation redundancy score